In [1]:
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
import pandas as pd

In [2]:
from sklearn.cross_validation import train_test_split

In [ ]:
# third trail

def words_generator(fileobj):
    for line in fileobj:
        for word in line.split():
            yield word

def make_Dictionary(root_dir):
    word_count_dict={}
    emails_dirs = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]    
    for emails_dir in emails_dirs:
        dirs = [os.path.join(emails_dir,f) for f in os.listdir(emails_dir)]
        for d in dirs:
            emails = [os.path.join(d,f) for f in os.listdir(d)]
            
            for mail in emails:
                file_name = mail.rsplit('/', 1)[-1]
                file_dict[file_name] = {}                    
                f = open(mail,"r")
                words = words_generator(f)
                for word in words:
                    if word.isalpha()==False:
                        continue                        
                    if word not in word_count_dict:
                          word_count_dict[word] = 0
                    word_count_dict[word]+= 1                    
    return word_count_dict


In [ ]:
def find_most_frequent_words(word_count_dict):
    #word_freq_dict = {}
    df = pd.DataFrame(word_count_dict.items(), columns=['word', 'total_freq'])
    #df['total_freq'] = df['freq_values'].apply(lambda x: x.get('total'))
    print len(df)
    df = df.sort(['total_freq'],ascending=False).head(n=3000)
    return df['word'].tolist()
    
    

In [ ]:
main_dir = "/home/datascience/Desktop/SpamFiltering/enronEmailData"

word_count_dict = make_Dictionary(main_dir)

freq_words = find_most_frequent_words(word_count_dict)

In [3]:
df = pd.read_csv('/home/datascience/Desktop/SpamFiltering/features.csv', sep=',', lineterminator='\n',
                 na_values=['-'])

In [4]:
columns = list(df.columns.values)
words = columns[:]
words.remove("is_spam")
words.remove("file_name")
words.remove("ind")

In [5]:
df_spam = pd.DataFrame()
df_spam["is_spam"] = df["is_spam"]
df.drop('is_spam', axis=1, inplace=True)

In [6]:
df.drop('file_name', axis=1, inplace=True)
df.drop('ind', axis=1, inplace=True)

In [7]:
df[pd.isnull(df).any(axis=1)]

,the,to,and,of,a,in,for,you,is,this,...,georgia,confident,asp,alliance,representatives,dealers,capabilities,eastrans,oh,addressed


In [7]:
X_train, X_test, y_train, y_test = train_test_split(df,df_spam, train_size = 0.6)


In [17]:
np.count_nonzero(y_test == 0)

6618

In [18]:
len(y_test)

13469

In [ ]:
from sklearn.svm import SVC
svc2 = SVC(kernel="poly")
svc2.fit(X_train,y_train)

svcresult2 = svc2.predict(X_test)
print confusion_matrix(y_test, svcresult2)


/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:441: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


In [40]:
model1 = LinearSVC()
model1.fit(X_train,y_train)

result1 = model1.predict(X_test)
print confusion_matrix(y_test, result1)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:125: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[6465  153]
 [ 128 6723]]


In [41]:
precision_recall_fscore_support(y_test,result1,pos_label=0)

(array([ 0.98058547,  0.97774869]),
 array([ 0.97688123,  0.9813166 ]),
 array([ 0.97872985,  0.97952939]),
 array([6618, 6851]))

In [8]:
from sklearn.naive_bayes import GaussianNB
gaussianclf = GaussianNB()
gaussianclf.fit(X_train,y_train)
gaussianclfresult = gaussianclf.predict(X_test)
print confusion_matrix(y_test, gaussianclfresult)

/usr/local/lib/python2.7/dist-packages/sklearn/naive_bayes.py:150: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[5831  787]
 [ 159 6692]]


In [30]:
# f1 score Gaussian NB
from sklearn.metrics import precision_recall_fscore_support, f1_score,precision_score
precision_recall_fscore_support(y_test,gaussianclfresult,pos_label=1)

(array([ 0.97345576,  0.89477203]),
 array([ 0.8810819 ,  0.97679171]),
 array([ 0.92496827,  0.93398465]),
 array([6618, 6851]))

In [25]:
precision_recall_fscore_support(y_test,gaussianclfresult,pos_label=0)

(array([ 0.97345576,  0.89477203]),
 array([ 0.8810819 ,  0.97679171]),
 array([ 0.92496827,  0.93398465]),
 array([6618, 6851]))

In [32]:
precision_score(y_test,gaussianclfresult,pos_label=1)

0.89477202834603553

In [33]:
from sklearn.naive_bayes import BernoulliNB
bernoulliclf = BernoulliNB()
bernoulliclf.fit(X_train,y_train)
bernoulliclfresult = bernoulliclf.predict(X_test)
print confusion_matrix(y_test, bernoulliclfresult)

/usr/local/lib/python2.7/dist-packages/sklearn/naive_bayes.py:301: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[5995  623]
 [  67 6784]]


In [35]:
precision_recall_fscore_support(y_test,bernoulliclfresult,pos_label=0)

(array([ 0.98894754,  0.91589037]),
 array([ 0.9058628 ,  0.99022041]),
 array([ 0.9455836 ,  0.95160612]),
 array([6618, 6851]))

In [36]:
model2 = MultinomialNB()
model2.fit(X_train,y_train)
result2 = model2.predict(X_test)
print confusion_matrix(y_test, result2)

/usr/local/lib/python2.7/dist-packages/sklearn/naive_bayes.py:301: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[6417  201]
 [ 142 6709]]


In [37]:
precision_recall_fscore_support(y_test,result2,pos_label=0)

(array([ 0.97835036,  0.97091172]),
 array([ 0.96962829,  0.9792731 ]),
 array([ 0.9739698 ,  0.97507449]),
 array([6618, 6851]))

In [42]:
# simple decision tree classifier

from sklearn import tree
decisionTree = tree.DecisionTreeClassifier()
decisionTree = decisionTree.fit(X_train,y_train)

decisionTreeResult = decisionTree.predict(X_test)
print confusion_matrix(y_test, decisionTreeResult)

[[6226  392]
 [ 339 6512]]


In [43]:
precision_recall_fscore_support(y_test,decisionTreeResult,pos_label=0)

(array([ 0.94836253,  0.94322132]),
 array([ 0.9407676 ,  0.95051817]),
 array([ 0.9445498 ,  0.94685569]),
 array([6618, 6851]))

In [21]:
# decision tree classifier variant (3 sec) 
decisionTree1 = tree.DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
decisionTree1 = decisionTree1.fit(X_train,y_train)

decisionTreeResult1 = decisionTree1.predict(X_test)
print confusion_matrix(y_test, decisionTreeResult1)

[[6195  394]
 [ 326 6554]]


In [39]:
# decision tree classifier variant (3 sec) 
from sklearn import tree
decisionTree1 = tree.DecisionTreeClassifier(max_depth=None, min_samples_split=2)
decisionTree1 = decisionTree1.fit(X_train,y_train)

decisionTreeResult1 = decisionTree1.predict(X_test)
print confusion_matrix(y_test, decisionTreeResult1)

[[6224  394]
 [ 349 6502]]


In [16]:

# 25 min
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3) 
neigh.fit(X_train, y_train) 
neighResult = neigh.predict(X_test)
print confusion_matrix(y_test, neighResult)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  app.launch_new_instance()


[[5626  963]
 [ 205 6675]]


In [17]:
# Random forest classifier (1 sec)
from sklearn.ensemble import RandomForestClassifier
randomForest = RandomForestClassifier(n_estimators=10)
randomForest = randomForest.fit(X_train, y_train)
randomForestResult = randomForest.predict(X_test)
print confusion_matrix(y_test, randomForestResult)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  app.launch_new_instance()


[[6386  203]
 [ 223 6657]]


In [44]:
# extra trees classifier (2 sec)
from sklearn.ensemble import ExtraTreesClassifier
extraTrees = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
extraTrees = extraTrees.fit(X_train, y_train)
extraTreesResult = extraTrees.predict(X_test)
print confusion_matrix(y_test, extraTreesResult)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[[6509  109]
 [ 255 6596]]


In [45]:
precision_recall_fscore_support(y_test,extraTreesResult,pos_label=0)

(array([ 0.96230041,  0.98374348]),
 array([ 0.98352977,  0.96277916]),
 array([ 0.97279928,  0.97314842]),
 array([6618, 6851]))

In [47]:
from sklearn.ensemble import RandomForestClassifier

randomForest1 = RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
randomForest1 = randomForest1.fit(X_train, y_train)
randomForestResult1 = randomForest1.predict(X_test)
print confusion_matrix(y_test, randomForestResult1)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[[6440  178]
 [ 221 6630]]


In [48]:
precision_recall_fscore_support(y_test,extraTreesResult,pos_label=0)

(array([ 0.96230041,  0.98374348]),
 array([ 0.98352977,  0.96277916]),
 array([ 0.97279928,  0.97314842]),
 array([6618, 6851]))

In [ ]:
import csv
your_file = "/home/datascience/words.csv"
with open(your_file, "r") as f:
    data = [x for x in csv.reader(f, delimiter='\t')]
with open(your_file, "w") as f:    
    w = csv.writer(f, delimiter=',')
    w.writerows(data)